In [ ]:
%load_ext blackcellmagic 
# %%black -l 120
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
from itertools import zip_longest
import numpy as np
import matplotlib.pyplot as plt
from experiments.base.compute_iqm import get_iqm_and_conf_per_epoch
from experiments.atari.exp_output import scale_score


games = ["BattleZone", "DoubleDunk", "NameThisGame"]
experiment_folders = {
    "activation": [
        "activation_tanh_relu_sigmoid_*/adadqnstatic",
        "lr_5e-5_*/dqn",
        "activation_tanh_*/dqn",
        "activation_sigmoid_*/dqn",
    ],
    "architecture": [
        "architecture_44_52_64_512__32_64_64_512__46_46_46_714_*/adadqnstatic",
        "lr_5e-5_*/dqn",
        "architecture_44_52_64_512_*/dqn",
        "architecture_46_46_46_714_*/dqn",
    ],
    "epsilon_adam": [
        "epsilon_adam_small_medium_large_*/adadqnstatic",
        "lr_5e-5_*/dqn",
        "epsilon_adam_medium_eps_*/dqn",
        "epsilon_adam_small_eps_*/dqn",
    ],
}

experiment_data = {
    experiment_type: {
        experiment: {game: {} for game in games + ["aggregate"]} for experiment in experiment_folders[experiment_type]
    }
    for experiment_type in experiment_folders
}

base_path = os.path.join(os.path.abspath(""), "static_hp_set")

def bincount_atleast3D(array):
    bincount = np.bincount(array)
    return np.append(bincount, np.zeros(3 - len(bincount)))

for experiment_type, list_experiments in experiment_folders.items():
    for experiment in list_experiments:
        raw_values = []
        raw_idx_count = []
        for game in games:
            experiment_path = os.path.join(base_path, experiment.replace("*", game), "episode_returns_and_lenghts")

            returns_experiment_ = []
            for experiment_file in os.listdir(experiment_path):
                list_episode_returns = json.load(open(os.path.join(experiment_path, experiment_file), "r"))[
                    "episode_returns"
                ]
                returns_experiment_.append(
                    [scale_score(np.mean(episode_returns), game) for episode_returns in list_episode_returns]
                )

            returns_experiment = np.array(list(zip_longest(*returns_experiment_, fillvalue=np.nan))).T

            print(f"Plot {experiment.replace('*', game)} with {returns_experiment.shape[0]} seeds.")
            n_epochs = json.load(open(os.path.join(experiment_path, "..", "..", "parameters.json"), "r"))["n_epochs"]
            if returns_experiment.shape[1] != n_epochs:
                print(f"!!! None of the seeds are not complete !!!")
            elif np.isnan(returns_experiment).any():
                seeds = np.array(list(map(lambda path: int(path.strip(".json")), os.listdir(experiment_path))))
                print(f"!!! Seeds {seeds[np.isnan(returns_experiment).any(axis=1)]} are not complete !!!")

            raw_values.append(returns_experiment)
            (
                experiment_data[experiment_type][experiment][game]["iqm"],
                experiment_data[experiment_type][experiment][game]["confidence"],
            ) = get_iqm_and_conf_per_epoch(returns_experiment)
            experiment_data[experiment_type][experiment][game]["x_values"] = np.arange(
                1, returns_experiment.shape[1] + 1
            )

            experiment_path_indices = os.path.join(
                base_path, experiment.replace("*", game), "indices_and_hyperparameters_details"
            )

            if os.path.exists(experiment_path_indices):
                indices_experiment_ = []
                for experiment_file in os.listdir(experiment_path_indices):
                    indices_experiment_.append(
                        json.load(open(os.path.join(experiment_path_indices, experiment_file), "r"))[
                                "compute_target"
                            ]
                    )
                nan_filled_indices_ = np.array(list(zip_longest(*indices_experiment_, fillvalue=np.nan))).T.astype(int)
                # split the indices in 40 epochs
                splitted_indices = nan_filled_indices_[:, : -(nan_filled_indices_.shape[1] % 40)].reshape(
                    (nan_filled_indices_.shape[0], 40, nan_filled_indices_.shape[1] // 40)
                )
                # count the number of occurances of each index
                indices_occurances = np.apply_along_axis(
                    bincount_atleast3D, 1, splitted_indices.transpose((1, 0, 2)).reshape((40, -1))
                )
                indices_frequency = indices_occurances.T / np.max(indices_occurances, axis=1)

                raw_idx_count.append(indices_occurances)
                experiment_data[experiment_type][experiment][game]["idx_target"] = indices_frequency

        (
            experiment_data[experiment_type][experiment]["aggregate"]["iqm"],
            experiment_data[experiment_type][experiment]["aggregate"]["confidence"],
        ) = get_iqm_and_conf_per_epoch(np.array(raw_values))
        experiment_data[experiment_type][experiment]["aggregate"]["x_values"] = np.arange(1, raw_values[0].shape[1] + 1)
        if len(raw_idx_count) > 0:
            idx_count = np.array(raw_idx_count).sum(axis=0)
            experiment_data[experiment_type][experiment]["aggregate"]["idx_target"] = idx_count.T / np.max(
            	idx_count, axis=1
            )

In [ ]:
from matplotlib.lines import Line2D
from experiments.atari.exp_output import NAMES, COLORS, STYLES, ORDERS


plt.rc("font", family="serif", serif="Times New Roman", size=22)
plt.rc("lines", linewidth=3)

fig, axes = plt.subplots(6, 3, figsize=(17, 10), height_ratios=[1.4, 2, 1.4, 2, 1.4, 2])

for idx_y_axis, experiment_type in enumerate(experiment_data):
    axes[2 * idx_y_axis, 1].text(
        0.5, 0.6, NAMES[experiment_type]["name"], horizontalalignment="center", verticalalignment="center", fontsize=24
    )

    for experiment in experiment_data[experiment_type]:
        for idx_x_axis, game in enumerate(games):
            axes[2 * idx_y_axis, idx_x_axis].axison = False
            hyperparameter, algorithm = experiment.split("_*/")

            ax = axes[2 * idx_y_axis + 1, idx_x_axis]
            ax.plot(
                experiment_data[experiment_type][experiment][game]["x_values"],
                experiment_data[experiment_type][experiment][game]["iqm"],
                color=COLORS[experiment_type][experiment],
                linestyle=STYLES[experiment_type][experiment],
                zorder=ORDERS[experiment_type][experiment],
            )
            ax.fill_between(
                experiment_data[experiment_type][experiment][game]["x_values"],
                experiment_data[experiment_type][experiment][game]["confidence"][0],
                experiment_data[experiment_type][experiment][game]["confidence"][1],
                color=COLORS[experiment_type][experiment],
                alpha=0.3,
                zorder=ORDERS[experiment_type][experiment],
            )

            ax.set_title(game)
            ax.grid(zorder=0)
            ax.set_xticks(range(0, 41, 10))

axes[5, 1].set_xlabel("Num Frames (in millions)", labelpad=10)
_ = axes[3, 0].set_ylabel("IQM Human Norm Score", labelpad=10)

In [ ]:
for idx_y_axis, experiment_type in enumerate(experiment_data):
    selected_legends = []
    selected_lines = []
    for experiment in experiment_data[experiment_type]:
        selected_legends.append(NAMES[experiment_type][experiment])
        selected_lines.append(
            Line2D([], [], color=COLORS[experiment_type][experiment], linestyle=STYLES[experiment_type][experiment])
        )

    axes[2 * idx_y_axis + 1, 1].legend(
        selected_lines,
        selected_legends,
        ncols=4,
        frameon=False,
        loc="center",
        bbox_to_anchor=(0.5, 1.4),
        columnspacing=0.7,
        fontsize=18,
    )

fig.subplots_adjust(wspace=0.2, hspace=0.3)
fig.savefig(f"{base_path}/performance_per_game.pdf", bbox_inches="tight")
fig

In [ ]:
from matplotlib.lines import Line2D
from experiments.atari.exp_output import NAMES, COLORS, STYLES, ORDERS


plt.rc("font", family="serif", serif="Times New Roman", size=22)
plt.rc("lines", linewidth=3)

fig, axes = plt.subplots(1, 3, figsize=(17, 2.5))

for idx_x_axis, experiment_type in enumerate(experiment_data):
    for experiment in experiment_data[experiment_type]:
        hyperparameter, algorithm = experiment.split("_*/")

        ax = axes[idx_x_axis]
        ax.plot(
            experiment_data[experiment_type][experiment]["aggregate"]["x_values"],
            experiment_data[experiment_type][experiment]["aggregate"]["iqm"],
            color=COLORS[experiment_type][experiment],
            linestyle=STYLES[experiment_type][experiment],
            zorder=ORDERS[experiment_type][experiment],
        )
        ax.fill_between(
            experiment_data[experiment_type][experiment]["aggregate"]["x_values"],
            experiment_data[experiment_type][experiment]["aggregate"]["confidence"][0],
            experiment_data[experiment_type][experiment]["aggregate"]["confidence"][1],
            color=COLORS[experiment_type][experiment],
            alpha=0.3,
            zorder=ORDERS[experiment_type][experiment],
        )

        ax.grid(zorder=0)
        ax.set_xticks(range(0, 41, 10))
        ax.set_xlabel("Num Frames (in millions)")

_ = axes[0].set_ylabel("IQM Human Norm Score")

In [ ]:
for idx_x_axis, experiment_type in enumerate(experiment_data):
    selected_legends = []
    selected_lines = []
    for experiment in experiment_data[experiment_type]:
        selected_legends.append(NAMES[experiment_type][f"{experiment}_short"])
        selected_lines.append(
            Line2D([], [], color=COLORS[experiment_type][experiment], linestyle=STYLES[experiment_type][experiment])
        )

    axes[idx_x_axis].legend(
        selected_lines,
        selected_legends,
        ncols=2,
        frameon=False,
        loc="center",
        bbox_to_anchor=(0.5, 1.17),
        columnspacing=0.7,
        fontsize=16.5,
    )
    axes[idx_x_axis].set_title(NAMES[experiment_type]["short_name"], pad=52)

fig.subplots_adjust(wspace=0.15)
fig.savefig(f"{base_path}/performance.pdf", bbox_inches="tight")
fig